### openmmlab 训练自己的光谱模型
第一步，更改dataset ， 因为原本数据格式不识别hdr，需要增加hdr。倘若是coco格式的标注文件，还需要更改coco标注文件的后缀为hdr。

第二步，需要更改继承LoadImageFromFile的方法，更改transform，使其能够读取光谱数据

第三步，需要更改数据预处理函数，使其能够处理光谱数据

#### 第一步，更改dataset 

两种更改方式，一种是直接在load_data_list，这个方法中直接添加后缀.hdr或者更改后缀

另外一种是增加可识别的后缀，增加hdr

In [ ]:
### 第一种方式

import os
import time
import tempfile
import copy
import traceback
import numpy as np
import os.path as osp
from typing import List, Union
from mmdet.datasets.coco import CocoDataset
from mmengine.fileio import get_local_path
from my_tools.mmdet.datasets.GXDetDataset import GXDetDataset
from mmseg.registry import DATASETS
from mmengine.dataset.base_dataset import Compose
from mmengine.logging import MMLogger
get_root_logger = MMLogger.get_current_instance
from my_tools.mmseg.piplines.init_pipeline import LoadCOCOStuffIAnnotations


@DATASETS.register_module()
class zwbSegDataset(GXDetDataset):

    def __init__( self):
      

    def load_data_list(self):

        try:
            with get_local_path(
                    self.ann_file,
                    backend_args=self.backend_args) as local_path:
                self.coco = self.COCOAPI(local_path)
            # The order of returned `cat_ids` will not
            # change with the order of the `classes`
            self.cat_ids = self.coco.get_cat_ids(
                cat_names=self.metainfo['classes'])
            self.cat2label = {
                cat_id: i
                for i, cat_id in enumerate(self.cat_ids)
            }
            self.cat_img_map = copy.deepcopy(self.coco.cat_img_map)

            img_ids = self.coco.get_img_ids()
            data_list = []
            total_ann_ids = []
            for img_id in img_ids:
                raw_img_info = self.coco.load_imgs([img_id])[0]
                raw_img_info['img_id'] = img_id

                ann_ids = self.coco.get_ann_ids(img_ids=[img_id])
                raw_ann_info = self.coco.load_anns(ann_ids)
                total_ann_ids.extend(ann_ids)

                parsed_data_info = self.parse_data_info({
                    'raw_ann_info':
                    raw_ann_info,
                    'raw_img_info':
                    raw_img_info
                })
                data_list.append(parsed_data_info)
            if self.ANN_ID_UNIQUE:
                assert len(set(total_ann_ids)) == len(
                    total_ann_ids
                ), f"Annotation ids in '{self.ann_file}' are not unique!"

            del self.coco
        except:
            pass

        for img_data in self.results['json_data_dic']['images']:
            if 'file_name' in img_data:
                img_data['file_name'] = img_data['file_name'].rsplit('.', 1)[0] + '.hdr'

        data_list = [{'img_path': img_data['file_name'].rsplit('.', 1)[0] + '.hdr',} for img_data in self.results['json_data_dic']['images']]

        return data_list

In [ ]:
### 第二种方式  在列表上面加上了hdr

# Copyright (c) OpenMMLab. All rights reserved.
from typing import List, Optional, Union

from mmengine import fileio
from mmengine.logging import MMLogger

from mmpretrain.registry import DATASETS
from mmpretrain.datasets import CustomDataset


@DATASETS.register_module()
class zwbImageNet(CustomDataset):
    """`ImageNet <http://www.image-net.org>`_ Dataset.

    The dataset supports two kinds of directory format,

    ::

        imagenet
        ├── train
        │   ├──class_x
        |   |   ├── x1.jpg
        |   |   ├── x2.jpg
        |   |   └── ...
        │   ├── class_y
        |   |   ├── y1.jpg
        |   |   ├── y2.jpg
        |   |   └── ...
        |   └── ...
        ├── val
        │   ├──class_x
        |   |   └── ...
        │   ├── class_y
        |   |   └── ...
        |   └── ...
        └── test
            ├── test1.jpg
            ├── test2.jpg
            └── ...

    or ::

        imagenet
        ├── train
        │   ├── x1.jpg
        │   ├── y1.jpg
        │   └── ...
        ├── val
        │   ├── x3.jpg
        │   ├── y3.jpg
        │   └── ...
        ├── test
        │   ├── test1.jpg
        │   ├── test2.jpg
        │   └── ...
        └── meta
            ├── train.txt
            └── val.txt


    Args:
        data_root (str): The root directory for ``data_prefix`` and
            ``ann_file``. Defaults to ''.
        split (str): The dataset split, supports "train", "val" and "test".
            Default to ''.
        data_prefix (str | dict): Prefix for training data. Defaults to ''.
        ann_file (str): Annotation file path. Defaults to ''.
        metainfo (dict, optional): Meta information for dataset, such as class
            information. Defaults to None.
        **kwargs: Other keyword arguments in :class:`CustomDataset` and
            :class:`BaseDataset`.


    Examples:
        >>> from mmpretrain.datasets import ImageNet
        >>> train_dataset = ImageNet(data_root='data/imagenet', split='train')
        >>> train_dataset
        Dataset ImageNet
            Number of samples:  1281167
            Number of categories:       1000
            Root of dataset:    data/imagenet
        >>> test_dataset = ImageNet(data_root='data/imagenet', split='val')
        >>> test_dataset
        Dataset ImageNet
            Number of samples:  50000
            Number of categories:       1000
            Root of dataset:    data/imagenet
    """  # noqa: E501

    IMG_EXTENSIONS = ('.hdr','.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', 'npy')
    # METAINFO = {'classes': IMAGENET_CATEGORIES}
    # METAINFO = {'classes': ('0', '1', '2', '3', '4', '5')}
    # METAINFO = {'classes': ('0', '1')}
    # METAINFO = {'classes': ('1', '4')}
    # METAINFO = {'classes': ('fsl', 'jy', 'qsdy', 'rhy', 'sdy')}
    # METAINFO = {'classes': ('ljz', 'jnz', 'you')}
    # METAINFO = {'classes': ('cjs', 'fsl', 'jy', 'qsdy', 'rhy', 'sdy', 'zyym')}
    # METAINFO = {'classes': ('cjs', 'fsl', 'jy', 'qsdy', 'rhy', 'sdy', 'zy')}
    # METAINFO = {'classes': ('cjs', 'fsl', 'jnz', 'jy', 'ljz', 'qsdy', 'rhy', 'sdy', 'zy')}
    # METAINFO = {'classes': ('jnz', 'ljz', 'no_jnz_ljz')}
    # METAINFO = {'classes': ('jnz_ljz', 'no_jnz_ljz')}
    # METAINFO = {'classes': ('shui', 'xiangjing', 'you')}
    # METAINFO = {'classes': ('fsl_jy_qsdy', 'rhy', 'sdy')}
    # METAINFO = {'classes': ('fsl_jy_qsdy', 'rhy', 'sdy', 'zy')}
    # METAINFO = {'classes': ('fsl', 'jy', 'qsdy', 'rhy_sdy_zy')}
    # METAINFO = {'classes': ('fsl', 'jy', 'qsdy')}
    # METAINFO = {'classes': ('cly', 'shl', 'rhy', 'rhz', 'sly', 'zcy')}
    METAINFO = {'classes': ('正常', '不正常')}

    def __init__(self,
                 data_root: str = '',
                 split: str = '',
                 data_prefix: Union[str, dict] = '',
                 ann_file: str = '',
                 metainfo: Optional[dict] = None,
                 **kwargs):
        kwargs = {'extensions': self.IMG_EXTENSIONS, **kwargs}

        if split:
            splits = ['train', 'val', 'test']
            assert split in splits, \
                f"The split must be one of {splits}, but get '{split}'"

            if split == 'test':
                logger = MMLogger.get_current_instance()
                logger.info(
                    'Since the ImageNet1k test set does not provide label'
                    'annotations, `with_label` is set to False')
                kwargs['with_label'] = False

            data_prefix = split if data_prefix == '' else data_prefix

            if ann_file == '':
                _ann_path = fileio.join_path(data_root, 'meta', f'{split}.txt')
                if fileio.exists(_ann_path):
                    ann_file = fileio.join_path('meta', f'{split}.txt')

        super().__init__(
            data_root=data_root,
            data_prefix=data_prefix,
            ann_file=ann_file,
            metainfo=metainfo,
            **kwargs)

    def extra_repr(self) -> List[str]:
        """The extra repr information of the dataset."""
        body = [
            f'Root of dataset: \t{self.data_root}',
        ]
        return body




#### 第二步需要更改继承LoadImageFromFile的方法，更改transform，使其能够读取光谱数据

In [ ]:

from mmcv.transforms import LoadImageFromFile
import spectral 
import numpy as np
from mmseg.registry import DATASETS, TRANSFORMS


@TRANSFORMS.register_module()
class LoadSpecralImageFromHdr(LoadImageFromFile):

    def transform(self, results: dict):

        filename = results['img_path']
        img = spectral.open_image(filename)[:,:,200:210]
        img = np.clip(img, 0, 255)  # Ensure values are within the valid range
        img = img.astype(np.uint8)   # Convert to uint8


        if self.to_float32:
            img = img.astype(np.float32)

        results['img'] = img
        results['img_shape'] = img.shape[:2]
        results['ori_shape'] = img.shape[:2]

        return results

#### 第三步，更改预处理方法，使其能够适配光谱图像多出的维度

一般预处理方法会有标准化的方法，一般标注化的方法是对RGB三个维度进行标准化，而对光谱图像进行标注化，就要增多相应的维度

并且，如有有转换bgr为rgb的操作，或者转换成hsv的操作，一定要找出并且取消掉。对光谱图像处理会报错

In [ ]:
##### 在配置文件中，进行如下更改
data_preprocessor = dict(
    type='SegDataPreProcessor',
    mean=[123.675]*10,   ### 扩增的10是光谱维度
    std=[58.395]*10,
    bgr_to_rgb=False,  ### 转换取消掉
    pad_val=0,
    seg_pad_val=255)


#### 第四步更改模型

倘若需要优化模型效果，肯定是需要改模型参数的，但是这里只介绍能够跑通的核心，修改输入层数

In [ ]:
model = dict(
    type='EncoderDecoder',
    data_preprocessor=data_preprocessor,
    pretrained='open-mmlab://resnet50_v1c',
    backbone=dict(
        type='ResNetV1c',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 2, 4),
        strides=(1, 2, 1, 1),
        norm_cfg=norm_cfg,
        norm_eval=False,
        style='pytorch',
        contract_dilation=True,
        in_channels=10)) ### 注意这里in_channels ，10是通道数

#### 第五步标注文件中的路径转换（路径转换为hdr为后缀）（coco数据集格式）

In [8]:
import json
import os

# Path to the folder containing your JSON files
folder_path = rf'H:\\临时使用数据\\高光谱大烟叶数据'

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Only process JSON files
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # Open and load the JSON file
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Now assuming 'imagePath' is at the first level of the dictionary
        if 'imagePath' in data:
            original_path = data['imagePath']
            try:
                # Change the file extension to .hdr
                new_path = os.path.splitext(original_path)[0] + '.hdr'
                data['imagePath'] = new_path
                print(f"Successfully changed path: {original_path} -> {new_path}")
            except Exception as e:
                print(f"Failed to change path for {original_path}. Error: {e}")

        # Save the modified JSON back to the file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        print(f"Processed {filename}")

Successfully changed path: newdata20250617_145549.png -> newdata20250617_145549.hdr
Processed newdata20250617_145549.json
Successfully changed path: newdata20250617_145711.png -> newdata20250617_145711.hdr
Processed newdata20250617_145711.json
